In [1]:
import os
import pickle
import numpy as np

from tqdm.notebook import tqdm # ui for data processing
from google.colab import drive

import tensorflow as tf
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.utils import load_img, img_to_array
from keras.models import Model

In [2]:
# mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Google Drive paths
GDRIVE_WORKING_PATH = "/content/gdrive/MyDrive/image_cap"
IMAGES_DIR = os.path.join(GDRIVE_WORKING_PATH, "iaprtc12/images")

## VGG16 on ImageNet
https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16

In [ ]:
if False:
  model = VGG16()

  # restructure model
  model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
  # we remove the last two layers because we don't need the final fully connected layers
  # of the vgg16, but only the layers that have the weights to extract the images features

  # summarize model
  model.summary()

In [ ]:
def extract_features_and_save(list_ids, name_pickle):
  features = {}
  for img_name in tqdm(list_ids):
    # load the image from file
    img_path = f"{IMAGES_DIR}/{img_name}.jpg"
    image = load_img(img_path, target_size=(224, 224))
    # convert image pxs to np array
    image = img_to_array(image)
    # reshape data for model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # preprocess image for vgg
    image = preprocess_input(image)
    # predict
    feature = model.predict(image, verbose=0)
    # store feature
    features[f"{img_name}"] = feature

  # store features in pickle
  pickle.dump(features, open(os.path.join(GDRIVE_WORKING_PATH, f"{name_pickle}.pkl"), "wb"))

In [ ]:
with open(f"{GDRIVE_WORKING_PATH}/list_id_train.pkl", "rb") as f:
  train_ids = pickle.load(f)
with open(f"{GDRIVE_WORKING_PATH}/list_id_test.pkl", "rb") as f:
  test_ids = pickle.load(f)
with open(f"{GDRIVE_WORKING_PATH}/list_id_val.pkl", "rb") as f:
  val_ids = pickle.load(f)

In [ ]:
print(f"Train has length {len(train_ids)} --> almost 70%")
print(f"Test has length {len(test_ids)} --> almost 20%")
print(f"Val has length {len(val_ids)} --> almost 10%")

Train has length 13988 --> almost 70%
Test has length 3996 --> almost 20%
Val has length 1999 --> almost 10%


In [ ]:
np.array(train_ids[:10])

array(['32811', '30644', '24393', '1176', '13858', '38876', '40226',
       '10304', '39401', '895'], dtype='<U5')

In [ ]:
np.array(test_ids[:10])

array(['39781', '18866', '16299', '2719', '31894', '7445', '13341',
       '12636', '31280', '38139'], dtype='<U5')

In [ ]:
np.array(val_ids[:10])

array(['10019', '11594', '40173', '15017', '3687', '17965', '19689',
       '33524', '31749', '14717'], dtype='<U5')

In [ ]:
if False:
  extract_features_and_save(train_ids, "dict_features_train_vgg16")
  extract_features_and_save(test_ids, "dict_features_test_vgg16")
  extract_features_and_save(val_ids, "dict_features_val_vgg16")

# VGG16 on Places365


In [ ]:
! sudo apt-get install python3-opencv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-pytest python3-numpy-dbg
The following NEW packages will be installed:
  python3-numpy python3-opencv
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 3,954 kB of archives.
After this operation, 21.6 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal/main amd64 python3-opencv amd64 4.2.0+dfsg-5+focal2 [1,230 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 python3-numpy amd64 1:1.17.4-5ubuntu3.1 [2,724 kB]
Fetched 3,954 kB in 1s (2,711 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend

In [10]:
import cv2
import numpy as np

In [ ]:
print(cv2.__version__)

4.7.0


In [ ]:
def extract_places_feature_and_save(net, list_ids, name_pickle):
  features = {}
  failed_ids = []
  for img_name in tqdm(list_ids):
    image_path = f"{GDRIVE_WORKING_PATH}/iaprtc12/images/{img_name}.jpg"
    image = cv2.imread(image_path)
    try:
      blob = cv2.dnn.blobFromImage(image, 1.0, (224, 224), (0.0, 0.0, 0.0), swapRB=True)

      # Set the input blob for the network
      net.setInput(blob)

      feature = net.forward("fc7")
      features[f"{img_name}"] = feature
    except Exception as e:
      failed_ids.append(img_name)

  # store features in pickle
  pickle.dump(features, open(os.path.join(GDRIVE_WORKING_PATH, f"{name_pickle}.pkl"), "wb"))
  print(failed_ids)

In [ ]:
# Set paths to the prototxt and caffemodel files
prototxt_file = f"{GDRIVE_WORKING_PATH}/places365/deploy_vgg16_places365.prototxt"
caffemodel_file = f"{GDRIVE_WORKING_PATH}/places365/vgg16_places365.caffemodel"

# Load the model
net = cv2.dnn.readNetFromCaffe(prototxt_file, caffemodel_file)

if True:
  extract_places_feature_and_save(net, train_ids, "dict_features_train_vgg16_places")
  extract_places_feature_and_save(net, test_ids, "dict_features_test_vgg16_places")
  extract_places_feature_and_save(net, val_ids, "dict_features_val_vgg16_places")

  0%|          | 0/13988 [00:00<?, ?it/s]

['19575', '19579', '17975']


  0%|          | 0/3996 [00:00<?, ?it/s]

[]


  0%|          | 0/1999 [00:00<?, ?it/s]

[]
